# Autodebugger with Watsonx.ai in Python

In this notebook we shows how to build this application. Currently the autodebugger is designed to run in python and try to do a recursive attempts to solve the code.

In [54]:
import os
import logging
import requests
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

load_dotenv()
def getBearer(apikey):
    form = {
        'apikey': apikey,
        'grant_type': "urn:ibm:params:oauth:grant-type:apikey"
    }
    logging.info("About to create bearer")
    response = requests.post(
        "https://iam.cloud.ibm.com/oidc/token",
        data=form
    )
    if response.status_code != 200:
        logging.error("Bad response code retrieving token")
        raise Exception("Failed to get token, invalid status")
    json = response.json()
    if not json:
        logging.error("Invalid/no JSON retrieving token")
        raise Exception("Failed to get token, invalid response")
    logging.info("Bearer retrieved")
    return json.get("access_token")


load_dotenv()
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.STOP_SEQUENCES: ["\n\n\n"]
}
project_id = os.getenv("PROJECT_ID", None)
credentials = {
    #"url": "https://eu-de.ml.cloud.ibm.com",
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}
credentials["token"] = getBearer(credentials["apikey"])
model_id = ModelTypes.LLAMA_2_70B_CHAT
# Initialize the Watsonx foundation model
llm_model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)


def generate_code(code, language, error):
    code_prompts = []
    code_prompt = f"""You are given a code snippet in {language} that contains syntax errors and logical issues. 
    Your task is to fix the code and provide the corrected version as the final result. 
    You should not provide any explanation or additional information; only the fixed code should be included in your response."""

    if error:
    
        inst_prompt = f"""<s>[INST] <<SYS>> 
            {code_prompt}
            <</SYS>> 
            The following is input code: {code}.
            The error is: {error}
            [/INST]  Answer only in {language} code:"""
        print("Prompt with error")
    else:
        
        inst_prompt = f"""<s>[INST] <<SYS>> 
            {code_prompt}
            <</SYS>> 
            The following is input code: {code}.
            [/INST]  Answer only in {language} code: """
        print("Prompt with no error")
    code_prompts.append(inst_prompt)
    logging.info("Sending prompt for code")
    result = llm_model.generate(
        code_prompts
    )
    code = ""
    for item in result:
        code += item['results'][0]['generated_text']
    logging.info("Obtained result")
    return code.strip()

def get_chatbot_suggestion(error, code):
    # Generate code fix using CodeGenerator
    generated_code = generate_code(code, "Python", error)
    return generated_code

In [55]:
error="""
  File "<stdin>", line 1
    "Hello world
    ^
SyntaxError: unterminated string literal (detected at line 1)

"""
code="""
"Hello world
"""

In [56]:
get_chatbot_suggestion(error, code)

Prompt with error


'"Hello world\\n"'

In [30]:
input_string = get_chatbot_suggestion(error, code)


In [41]:
import subprocess
from utils import *
def run_code(code):
    result = subprocess.run(["python", "-c", code], capture_output=True, text=True)
    if result.returncode == 0:
        return True, result.stdout
    else:
        return False, result.stderr

In [42]:
def debug_code(code_input):
    code = code_input
    attempt = 1
    success=False
    while success == False and attempt < 3:
        #output_zone.write(f"Attempt {attempt}: Running code...")
        print(f"Attempt {attempt}: Running code...")
        success, output = run_code(code)
        if success:
            #output_zone.write(f"Code executed successfully. Output: {output}")
            print((f"Code executed successfully. Output: {output}"))
            break
        else:
            error = output
            suggestion = get_chatbot_suggestion(error, code)  # Use your chatbot API to get the suggestion
            code = suggestion  # Update the code with the fixed version from the chatbot
            #fixed_code.write(f"Suggested fix applied:\n{code}")
            print(f"Suggested fix applied:\n{code}")
            #output_zone.write(f"Trying again with the fixed code...")
            print(f"Trying again with the fixed code...")
            attempt += 1   

In [57]:
code="""
print("Hello world)
"""

In [52]:
success, output = run_code(code)

In [53]:
output

'  File "<string>", line 2\r\n    print("Hello world)\r\n                       ^\r\nSyntaxError: EOL while scanning string literal\r\n'

In [47]:
run_code(code)

(True, 'Hello world\r\n')

In [48]:
debug_code(code)

Attempt 1: Running code...
Code executed successfully. Output: Hello world



In [59]:
if code :
    print(" no code")
    

 no code


In [61]:
code='''

# Python Program to convert temperature in celsius to fahrenheit

# change this value for a different result
celsius = 37.5

# calculate fahrenheit
fahrenheit = (celsius x 1.8) + 32
print('%0.1f degree Celsius is equal to %0.1f degree Fahrenheit' %(celsius,fahrenheit))

'''


In [60]:
def run_code(code):
    try:
        output = subprocess.check_output(["python", "-c", code])
        return True, output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        error = str(e.output.decode("utf-8"))
        return False, error   
    
success, output = run_code(code)    

In [63]:
success

False

In [64]:
output

''

In [65]:
def run_code(code):
    result = subprocess.run(["python", "-c", code], capture_output=True, text=True)
    if result.returncode == 0:
        return True, result.stdout
    else:
        return False, result.stderr

In [66]:
success, output = run_code(code)    

In [67]:
success

False

In [68]:
output

'  File "<string>", line 9\n    fahrenheit = (celsius x 1.8) + 32\n                          ^\nSyntaxError: invalid syntax\n'

In [69]:
code='''

# Python Program to convert temperature in celsius to fahrenheit

# change this value for a different result
celsius = 37.5

# calculate fahrenheit
fahrenheit = (celsius * 1.8) + 32
print('%0.1f degree Celsius is equal to %0.1f degree Fahrenheit' %(celsius,fahrenheit))

'''


In [70]:
success, output = run_code(code)   

In [71]:
success

True

In [72]:
output

'37.5 degree Celsius is equal to 99.5 degree Fahrenheit\n'